# 0. Import Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import date, timedelta
import random
import threading
import numpy
import multiprocessing as np
from multiprocessing import Pool
import concurrent.futures
import os

# 1. Functions

## 1.1. Loop Through Dates

In [2]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

In [3]:
# driver = webdriver.Chrome()
# driver.get("https://www.politico.com/search/20?adv=true&start=02/01/2016&end=02/01/2016")
# #print(driver.find_element(By.CSS_SELECTOR, "a.button").text)
# print(driver.find_element(By.CSS_SELECTOR, "a.button:nth-child(2)").text)

# 2. Scraping News Media

## 2.1. Wall Street Journal

### 2.1.1. Function - Get URLS

In [4]:
def getWallStreetJournalURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    #Define list of variables to store 
    WSJ_date = []
    WSJ_link = []

    #Go to the website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    driver_WSJ = webdriver.Firefox()
    driver_WSJ.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20createTimestamp&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=ab6a473a-cfa6-4714-8fad-b6dff98f5f18&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=8rChOTDzC_Y_AK-i.TJAixN_XjsWxwUEEPoHg2OPCaX6qRBu4nGSk5fqLliY4H0B5F7gj_57-XH-YBWGS&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")

    #put in user_name:
    time.sleep(3) #makes sure field is fully loaded
    username_field = driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[1]/div[2]/input")
    username_field.send_keys(username)

    #continue to password:
    driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[6]/div[1]/button[2]").click()

    #input password:
    time.sleep(3) #makes sure field is fully loaded
    password_field = driver_WSJ.find_element(By.XPATH, "//*[@id='password-login-password']")
    password_field.send_keys(password)

    #click on sign in:
    driver_WSJ.find_element(By.XPATH, "//*[@id='password-login']/div/form/div/div[5]/div[1]/button").click()

    try:
        #accept cookies
        time.sleep(10)
        driver_WSJ.switch_to.frame(WebDriverWait(driver_WSJ,30).until(EC.presence_of_element_located((By.ID, 'sp_message_iframe_718122'))))
        driver_WSJ.find_element(By.CSS_SELECTOR, "button.message-component:nth-child(2)").click()
        driver_WSJ.switch_to.default_content() 
        time.sleep(3)
    except:
        pass

    scrape_time = time.time() #starting the timing

    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%d/%m/%Y")
        print(date)
        year = "%02d" % (int(date.split('/')[2]),)
        month = "%02d" % (int(date.split('/')[1]),)
        day = "%02d" % (int(date.split('/')[0]),)
        page = "%02d" % (1,) #we assume there is always at least one page
        url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
        test = 0
        while(test < 100):
            try:
                driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
                break
            except:
                test += 1
                time.sleep(10)

        #check if there are multiple pages and if so, visit these as well
        pages_load = 0
        while(pages_load < 100):
            try:
                pages = WebDriverWait(driver_WSJ, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "WSJTheme--pagepicker-total--Kl350I1l ")))
                page_total = re.findall(r'\d+', pages.text)
                nr_pages = int(page_total[0])
                break
            except:
                pages_load += 1
                driver_WSJ.get(url)

        for p in range(1,nr_pages+1):
            page = "%02d" % (p,)
            url = f"https://www.wsj.com/news/archive/{year}/{month}/{day}?page={page}"
            urls_load = 0
            while(urls_load < 100):
                try:
                    driver_WSJ.get(url) #go to the page in the WSJ archive for the given date
                    break
                except:
                    driver_WSJ.get(url)

            #get the article urls for the articles on the other pages, if there are more than 1
            articles = WebDriverWait(driver_WSJ,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
            for a in range(0,len(articles)):
                WSJ_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                WSJ_date.append(date)
    
    driver_WSJ.quit()
    
    WSJ = ["Wall_Street_Journal"] * len(WSJ_link)
    data = {"Date" : WSJ_date, "News Paper" : WSJ, "Link" : WSJ_link}
    Wall_Street_Journal = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Wall_Street_Journal_{start_date}_{end_date}_URLS"
    Wall_Street_Journal.to_parquet(path) 
    
    print("--- %s seconds ---" % (time.time() - scrape_time))

### 2.1.2. Function - Articles

In [ ]:
def getWallStreetJournalArticles(username, password, dataframe):
    WSJ_article_content = []
    WSJ_title = []

    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)
    
    #CHROME
    # chrome_options = Options()
    # chrome_options.add_argument("--window-size=1920x1080")
    # chrome_options.add_argument("--disable-extensions")
    # chrome_options.add_argument("--disable-gpu")
    # chrome_options.add_argument("--no-sandbox") # linux only
    # #chrome_options.add_argument("--headless")
    # user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    # chrome_options.add_argument(f'user-agent={user_agent}')
    # driver_WSJ = webdriver.Chrome(options=chrome_options)
    driver_WSJ = webdriver.Chrome()
    driver_WSJ.get("https://sso.accounts.dowjones.com/login-page?op=localop&scope=openid%20idp_id%20roles%20email%20given_name%20family_name%20djid%20djUsername%20djStatus%20trackid%20tags%20prts%20suuid%20updated_at&client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&response_type=code&redirect_uri=https%3A%2F%2Faccounts.wsj.com%2Fauth%2Fsso%2Flogin&nonce=5e93fe71-314a-44b9-bb2d-4a0e5a3167a4&ui_locales=en-us-x-wsj-223-2&mars=-1&ns=prod%2Faccounts-wsj&state=zJgHWictetlLHLG6.Uob1DvxnFA4kIkiqJAY80zp2N4FxS3WdBvV-VuTFdpM&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin")

    #put in user_name:
    time.sleep(3) #makes sure field is fully loaded
    username_field = driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[1]/div[2]/input")
    username_field.send_keys(username)

    #continue to password:
    driver_WSJ.find_element(By.XPATH, "/html/body/div[1]/div[2]/div/div/div/div[1]/div[1]/form/div[2]/div[6]/div[1]/button[2]").click()

    #input password:
    time.sleep(3) #makes sure field is fully loaded
    password_field = driver_WSJ.find_element(By.XPATH, "//*[@id='password-login-password']")
    password_field.send_keys(password)

    #click on sign in:
    driver_WSJ.find_element(By.XPATH, "//*[@id='password-login']/div/form/div/div[5]/div[1]/button").click()

    try:
        #accept cookies
        time.sleep(20)
        driver_WSJ.switch_to.default_content()
        driver_WSJ.switch_to.frame(WebDriverWait(driver_WSJ,30).until(EC.presence_of_element_located((By.ID, 'sp_message_iframe_718122'))))
        driver_WSJ.find_element(By.CSS_SELECTOR, "button.message-component:nth-child(2)").click()
        print("accepted")
        time.sleep(3)
    except:
        pass

    scrape_time = time.time()
    #go to all the articles and scrape the content
    links  = list(dataframe["Link"])
    for u in range(0, len(links)):
        trys = 0
        while(trys < 1):
            try:
                driver_WSJ.get(links[u])
                break
            except:
                trys += 1
                time.sleep(10)

        try:
            content = driver_WSJ.find_element(By.TAG_NAME, 'article') 
            #extract the content and add to a variables
            text = content.find_elements(By.TAG_NAME, 'p')
            article_text = ""
            for t in range(0,len(text)):
                article_text += " " + text[t].text

            WSJ_article_content.append(article_text) #add text to the list
            #WSJ_article_content.append(content.text) #add text to the list

        except: #we don't have access to the article
            WSJ_article_content.append("NO ACCESS")
            pass #go back to page with all articles

        try:
            #collect the title as well
            title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.TAG_NAME, "h1"))).text
            WSJ_title.append(title)
        except:
            try:
                #there are 2 main formats in which the titles are present in the HTML
                title = WebDriverWait(driver_WSJ,10).until(EC.presence_of_element_located((By.CLASS_NAME, "bigTop__hed"))).text
                WSJ_title.append(title)

            except:
                #add error if there is some unexpected layout -> this way our arrays will be of the same length and we will be able
                #to construct a dataframe in the end
                WSJ_title.append("ERROR")
                pass

    driver_WSJ.quit()

    WSJ = ["Wall_Street_Journal"] * len(WSJ_article_content)
    data = {"Title" : WSJ_title, "Text" : WSJ_article_content, "Link" : dataframe["Link"]}
    Wall_Street_Journal = pd.DataFrame(data)
    Wall_Street_Journal = pd.merge(Wall_Street_Journal, dataframe, on = "Link")
    
    print("--- %s seconds ---" % (time.time() - scrape_time))
    return(Wall_Street_Journal)


## 2.2. Washington Post

### 2.2.1. Function - Get URLS

In [6]:
def getWashingtonPostURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y")   
    end_date = datetime.strptime(end_date, "%d/%m/%Y")


    #put them in correct format


    #Declare all list variables for the output
    WP_date = []
    WP_link = []
    
    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver = webdriver.Firefox(options=fireFoxOptions)
    
    #CHROME
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver = webdriver.Chrome()
    # Initialize the driver


    #Go the the searchpage of the Washington Post
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")
    #Accept Cookies
    try:
        driver.save_screenshot("cookies.png")
        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        pass
    
    #Log-in into Washington Post account
    try: #it is possible that account is already logged in! 
        #Sign in into Washington Post account
        #Click on Sign In
        #WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div[1]/nav/div[4]/div[2]/a/p"))).click()
        driver.get("https://www.washingtonpost.com/subscribe/signin/?next_url=https%3A%2F%2Fwww.washingtonpost.com&nid=top_pb_signin&arcId=&itid=nav_sign_in")
        time.sleep(5)
        driver.save_screenshot("Login.png")
        #located username field
        time.sleep(2) #field needs to completely load
        username_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='username']")))
        username_field.send_keys(username) #send username
        username_field.send_keys(Keys.RETURN) #press enter

        #Located password field
        time.sleep(2) #field needs to completely load
        password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='password']")))
        password_field.send_keys(password)
        password_field.send_keys(Keys.ENTER)

        #Go back to the search page
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "wpds-c-jlBemH "))).click()
        go_to_search = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "query")))
        go_to_search.send_keys(search_term)
        go_to_search.send_keys(Keys.ENTER)

    except:
        #you already logged-in -> continue 
        pass
    
    search_page_load = 0
    while(search_page_load < 100):
        try:
            driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")
            break
        except:
            search_page_load += 1
            
    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%y")

        #Locate the searchbar, send searchterm and press enter
        
        time.sleep(10)
        driver.save_screenshot("Searchbar.png")
        searchbar = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, "aa-Input")))
        searchbar.send_keys(" ")
        searchbar.send_keys(Keys.ENTER)

        #Select time period
        #select periode specification
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/button/div/div[2]/span"))).click()

        #select custom time
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[1]/button/div/div[2]"))).click()
        time.sleep(1)

        #Send start date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[1]/div/div/div[1]/input"))).send_keys(date)

        #Empty automatic end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).clear()

        #Send end date
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input"))).send_keys(date)

        #Press Apply
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[2]/div[1]/div/div/div[2]/button[2]"))).click()
        time.sleep(5)
        driver.refresh()
        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='main-content']/div[1]/section[3]/button"))).click()
            except:
                loading = False
        
        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))

        for a in range(0,len(articles)):
            WP_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            WP_date.append(single_date.strftime("%d/%m/%Y"))
    
    driver.quit()
    
    WP = ["Washington_Post"] * len(WP_link)
    data = {"Date" : WP_date, "News Paper" : WP, "Link" : WP_link}
    WashingtonPost = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Washington_Post_{start_date}_{end_date}_URLS"
    #WashingtonPost.to_parquet(path) 
    return(WashingtonPost)

    print("--- %s seconds ---" % (time.time() - scrape_time))

### 2.2.2. Function - Articles

In [7]:
def getWashingtonPostArticles(username, password, dataframe):
    WP_article_content = []
    WP_title = []

    #FIREFOX
    #Declare the driver and go to website
    #fireFoxOptions = webdriver.FirefoxOptions()
    #fireFoxOptions.add_argument('--ignore-certificate-errors')
    #fireFoxOptions.add_argument('--allow-running-insecure-content')
    #fireFoxOptions.add_argument("--headless")
    #fireFoxOptions.add_argument("--disable-gpu")
    #fireFoxOptions.add_argument("--no-sandbox")
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    #fireFoxOptions.add_argument(f'user-agent={user_agent}')
    #driver_WSJ = webdriver.Firefox(options=fireFoxOptions)

    #CHROME
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    #chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome()

    #Go the the searchpage of the Washington Post
    driver.get("https://www.washingtonpost.com/search/?query=+&facets=%7B%22time%22%3A%22all%22%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%5D%2C%22author%22%3A%5B%5D%7D")

    try:
        #Accept Cookies
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        print("no cookies")
        pass
    #Log-in into Washington Post account
    try: #it is possible that account is already logged in! 

        #Sign in into Washington Post account
        #Click on Sign In
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='__next']/div[1]/nav/div[4]/div[2]/a/p"))).click()
        print('signin attempted')
        #located username field
        time.sleep(2) #field needs to completely load
        username_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='username']")))
        username_field.send_keys(username) #send username
        username_field.send_keys(Keys.RETURN) #press enter

        #Located password field
        time.sleep(2) #field needs to completely load
        password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='password']")))
        password_field.send_keys(password)
        password_field.send_keys(Keys.ENTER)

        time.sleep(10)

    except:
        #you already logged-in -> continue 
        pass

    scrape_time = time.time()
    links = list(dataframe["Link"])
    for u in range(0, len(links)):
            trys = 0
            while(trys < 100):
                try:
                    #go to every article
                    driver.get(links[u])
                    break
                except:
                    trys += 1
            try:
                text = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.TAG_NAME, "article"))).text
            except:
                text = ""
                
            try:
                text = text[text.index("Share")+len("Share")+1:text.rindex("Gift Article")]
                WP_article_content.append(text)
            except:
                try:
                    text = text[text.index("Share")+len("Share")+1:text.rindex("Comments")]
                    WP_article_content.append(text)
                except:
                    try:
                        text = text[text.index("tb.boedttibo")+len("tb.boedttibo")+1:text.rindex("Comments")]
                        WP_article_content.append(text)
                    except:
                        WP_article_content.append(text)
                        pass
            #idea is: We take the full text and before we have the article content the last pice of text is "share" and immediatly after the content
            #we have "comments" this way we are able to only extract the article content
            #the reasons for this is that each article has a more or less different html dependent on their "type"

            #there are different formats of titles, in this order all titles are located
            try:
                title = driver.find_element(By.TAG_NAME, "h1").text
                WP_title.append(title)
            except:
                WP_title.append("ERROR")
                pass

    #Close driver
    driver.quit()

    #How long did the scraping took? 
    print("--- %s seconds ---" % (time.time() - scrape_time))

    #Create output file
    #Make the dataframe
    WP = ["Washington Post"] * len(WP_article_content)
    data = {"Title" : WP_title, "Text" : WP_article_content, "Date" : dataframe["Date"], "News Paper" : dataframe["News Paper"], "Link" : dataframe["Link"]}
    WashingtonPost = pd.DataFrame(data)
    return(WashingtonPost)

## 2.3. New York Times

### 2.3.1. Function - Get URLS

In [8]:
def getNewYorkTimesURLS(start_date, end_date, username, password):
    #put dates in date format
    start_date = datetime.strptime(start_date, "%d/%m/%Y") 
    end_date = datetime.strptime(end_date, "%d/%m/%Y")

    #Declare all list variables for the output
    NYT_date = []
    NYT_link = []

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox") # linux only
    chrome_options.add_argument("--headless")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fonboarding-offer%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead")
    time.sleep(5)
    driver.save_screenshot("test.png")
    #not a robot
    input("Press Enter to continue...")

    #Select log-in field
    login = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "email")))

    #Send username
    login.send_keys(username)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > div > form > div > div.css-bho3kg-buttonWrapper-buttonStyles-Button > button"))).click()

    #Select password field
    password_field = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "password")))

    #send password
    password_field.send_keys(password)

    #Press continue
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#myAccountAuth > div > div > form > div > div.css-1nkv26b-buttonWrapper-buttonStyles-Button > button"))).click()

    time.sleep(1)

    #go to search page
    search_page_loaded = 0
    while(search_page_loaded < 100):
        try:
            driver.get("https://www.nytimes.com/search?dropmab=false&query=&sort=best")
            break
        except:
            search_page_loaded += 1

    #Accept Cookies
    try:
        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME, "banner__container__cta--accept"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%Y%m%d")

        url = f"https://www.nytimes.com/search?dropmab=false&endDate={date}&query=%20&sort=best&startDate={date}"
        url_loaded = 0
        while(url_loaded < 100):
            try:
                driver.get(url)
                break
            except:
                url_loaded += 1

        #click to load more on the page untill no longer possible
        loading = True
        while(loading):
            try:
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='site-content']/div/div[2]/div[2]/div/button"))).click()
            except:
                loading = False

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-1l4w6pd")))
        dates = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))

        date_list = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-17ubb9w")))
        true_date = date_list[0].text
        for a in range(0, len(articles)):
            if(true_date == date_list[a].text):
                NYT_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
                NYT_date.append(single_date.strftime("%d/%m/%Y"))
    
    driver.quit()
    
    NYT = ["New_York_Times"] * len(NYT_link)
    data = {"Date" : NYT_date, "News Paper" : NYT, "Link" : NYT_link}
    New_York_Times = pd.DataFrame(data)
    
    print("--- %s seconds ---" % (time.time() - scrape_time))
    
    return(New_York_Times)
    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/New_York_Times_{start_date}_{end_date}_URLS"
    New_York_Times.to_parquet(path)

## 2.4. Politico

### 2.4.1. Function - Get URLS

In [9]:
def getPolitico(start_date, end_date):
    #put dates in date format
    end_date = datetime.strptime(end_date, "%d/%m/%Y")
    start_date = datetime.strptime(start_date, "%d/%m/%Y")

    #Declare all list variables for the output
    P_date = []
    P_link = []

    fireFoxOptions = webdriver.FirefoxOptions()
    fireFoxOptions.add_argument('--ignore-certificate-errors')
    fireFoxOptions.add_argument('--allow-running-insecure-content')
    fireFoxOptions.add_argument("--headless")
    fireFoxOptions.add_argument("--disable-gpu")
    fireFoxOptions.add_argument("--no-sandbox")
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    fireFoxOptions.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Firefox(options=fireFoxOptions)

    #go to the website
    driver.get("https://www.politico.com/search?adv=true")

    #accept cookies
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()
    except:
        pass

    scrape_time = time.time()
    for single_date in daterange(start_date, end_date):
        date = single_date.strftime("%m/%d/%Y")
        test = 0
        while(test < 100):
            try:
                driver.get(f"https://www.politico.com/search/1?adv=true&start={date}&end={date}")
                break
            except:
                test += 1
                time.sleep(10)
        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(date)

        #get the number of pages to know how much you should click .pagination > ol:nth-child(2) > li:nth-child(3) > a:nth-child(1)
        try:
            nr_pages = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pagination > ol:nth-child(2) > li:nth-child(6) > a:nth-child(1)"))).text
        except:
            nr_pages = 1
            pass

        for p in range(1, int(nr_pages)):
            url = f"https://www.politico.com/search/{p+1}?adv=true&start={date}&end={date}"
            driver.get(url)

        articles = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "article")))
        for a in range(0, len(articles)):
            P_link.append(articles[a].find_element(By.TAG_NAME, "a").get_attribute('href'))
            P_date.append(single_date.strftime("%d/%m/%Y"))

    driver.quit()
    print("--- %s seconds ---" % (time.time() - scrape_time))

    P = ["Politico"] * len(P_link)
    data = {"Date" : P_date, "News Paper" : P, "Link" : P_link}
    Politico = pd.DataFrame(data)

    start_date = start_date.strftime("%d%m%Y")
    end_date = end_date.strftime("%d%m%Y")
    path = f"Data/URLS/Politico_{start_date}_{end_date}_URLS"
    Politico.to_parquet(path)

# 3. Running Functions

## 3.1. Wall Street Journal

## 3.2. Washington Post

In [10]:
WP_URLS = pd.read_parquet("Washington_Post_2017urls_to_scrape").reset_index(drop = True)

In [11]:
# #URLS_WP = pd.read_parquet("Washington_Post_2020_URLS")
# year="2016"
# URLS_WP = pd.read_parquet("Wall_Street_Journal_"+year+"_URLS(1)")
# URLS_WP

In [12]:
URLS_WP = pd.read_parquet("Wall_Street_Journal_2017urls_to_scrape")
URLS_WP

,Link,Title,Text,Date,News Paper
0,http://www.wsj.com/articles/photos-of-the-day-...,Photos of the Day: Jan. 1,NO ACCESS,01/01/2017,Wall_Street_Journal
1,http://www.wsj.com/articles/chicago-marchers-m...,Chicago Marchers Mark Year of Violence,NO ACCESS,01/01/2017,Wall_Street_Journal
2,http://www.wsj.com/articles/re-energized-dolla...,Re-Energized Dollar Looms Over the Rest of the...,Write to Ira Iosebashvili a...,01/01/2017,Wall_Street_Journal
3,http://www.wsj.com/articles/new-jersey-town-ca...,New Jersey Town Calls on Uber to Solve Commute...,Write to Esther Fung at esther.fun...,01/01/2017,Wall_Street_Journal
4,http://www.wsj.com/articles/hawaiis-homeless-p...,"Hawaii’s Homeless Problem Spurs Fixes, and a D...",Write to Alejandro Lazo at...,01/01/2017,Wall_Street_Journal
...,...,...,...,...,...
38252,https://www.wsj.com/articles/funding-snapshot-...,Funding Snapshot: Mobile Video Startup LoopMe ...,NO ACCESS,07/03/2017,Wall_Street_Journal
38253,https://www.wsj.com/articles/sendgrid-acquires...,SendGrid Acquires Bizzy After Raising $33 Million,NO ACCESS,07/03/2017,Wall_Street_Journal
38254,https://www.wsj.com/articles/funding-snapshot-...,Funding Snapshot: Fintech Startup Current Clos...,NO ACCESS,07/03/2017,Wall_Street_Journal
38255,https://www.wsj.com/articles/funding-snapshot-...,Funding Snapshot: Farmer’s Business Network Se...,NO ACCESS,07/03/2017,Wall_Street_Journal


In [13]:
# update_2017 = getWashingtonPostArticles("tb.boedttibo@gmail.com", "ThesisR&T", WP_URLS[1:250])

In [14]:
# URLS_WP

In [15]:
# year = "2016"
# source = 'Washington_Post_'
# #source = "Wall_Street_Journal_"
# ### check the most common results for error indicators
# df = pd.read_parquet(source+year+"_Articles")
# # print(df.size)
# # pd.DataFrame(df['Text'].value_counts())

# to_re_scrape = df[df['Text']== ''].reset_index().drop(['index'], axis=1)
# URLS_WP = to_re_scrape
# URLS_WP

In [16]:
tijd = time.time()
n=1
# rescrape 3
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    while(n < 14):
        k = 1000*n
        # Submit the two instances of the scraping function to the executor
        future1 = executor.submit(getWallStreetJournalArticles, "reane.delaunoy@telenet.be", "REenJUC0MB0", URLS_WP[k+1:k+250])
        future2 = executor.submit(getWallStreetJournalArticles, "reane.delaunoy@telenet.be", "REenJUC0MB0", URLS_WP[k+251:k+500])
        future3 = executor.submit(getWallStreetJournalArticles, "reane.delaunoy@telenet.be", "REenJUC0MB0", URLS_WP[k+501:k+750])
        future4 = executor.submit(getWallStreetJournalArticles, "reane.delaunoy@telenet.be", "REenJUC0MB0", URLS_WP[k+751:k+1000])
        # future1 = executor.submit(getWashingtonPostArticles,  "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[k+1:k+250])
        # future2 = executor.submit(getWashingtonPostArticles,  "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[k+251:k+500])
        # future3 = executor.submit(getWashingtonPostArticles,  "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[k+501:k+750])
        # future4 = executor.submit(getWashingtonPostArticles,  "tb.boedttibo@gmail.com", "ThesisR&T", URLS_WP[k+751:k+1000])
        # Wait for the scraping functions to complete
        concurrent.futures.wait([future1, future2, future3, future4])

        print("--------------------------")   
        print("--- %s seconds ---" % (time.time() - tijd))

        result1= future1.result()
        result2= future2.result()
        result3= future3.result()
        result4= future4.result()
        frames = [result1, result2, result3, result4]
        df = pd.concat(frames).reset_index(drop = True)
        articles = pd.read_parquet("Wall_Street_Journal_2017_re-scraped") 
        articles = pd.concat([articles, df]).reset_index(drop = True)
        articles.to_parquet("Wall_Street_Journal_2017_re-scraped")
        #articles = pd.read_parquet("Washington_Post_"+year+"_Articles")
        #articles = pd.concat([articles, df]).reset_index(drop = True)
        #articles.to_parquet("Washington_Post_"+year+"_Articles")
        n=n+1
        print("--------------------------")   
        print(n)
        print("--------------------------")   


accepted
accepted
accepted
accepted
--- 649.023964881897 seconds ---
--- 966.1677751541138 seconds ---
--- 994.9101903438568 seconds ---
--- 1090.457921743393 seconds ---
--------------------------
--- 1126.9938366413116 seconds ---
--------------------------
2
--------------------------
accepted
accepted
accepted
accepted


In [ ]:

df.to_parquet("WSJ_missed_articles")


result1= future1.result()
result2= future2.result()
result3= future3.result()
result4= future4.result()
frames = [result1, result2, result3, result4]
df = pd.concat(frames).reset_index(drop = True)
articles = pd.read_parquet("Washington_Post_2016_Articles")
articles = pd.concat([articles, df]).reset_index(drop = True)
articles.to_parquet("Washington_Post_2016_Articles")

In [22]:
# which to re-scrape
year = "2016"
#source = 'Washington_Post_'
source = "Washington_Post_"
original_articles = pd.read_parquet(source+year+"_Articles")
articles = pd.read_parquet(source+year+"_Articles")
urls= pd.read_parquet(source+year+"_URLS")

articles['Text'].replace('NO ACCESS', numpy.nan, inplace=True)
articles['Text'].replace('404', numpy.nan, inplace=True)
articles.dropna(subset=['Text'], inplace=True)

df = pd.merge(urls, articles[['Link']], on=['Link','Link'], how="outer", indicator=True
              ).query('_merge=="left_only"').reset_index()
to_re_scrape = df.loc[:, df.columns != '_merge']

URLS_WP =to_re_scrape
URLS_WP


,index,Date,News Paper,Link
0,542,05/01/2016,Washington_Post,https://www.washingtonpost.com/news/dr-gridloc...
1,792,06/01/2016,Washington_Post,https://www.washingtonpost.com/opinions/whos-r...
2,1042,08/01/2016,Washington_Post,https://www.washingtonpost.com/news/the-switch...
3,1292,11/01/2016,Washington_Post,https://www.washingtonpost.com/local/i-despise...
4,1542,12/01/2016,Washington_Post,https://www.washingtonpost.com/news/early-lead...
...,...,...,...,...
193,48792,21/12/2016,Washington_Post,https://www.washingtonpost.com/news/true-crime...
194,49042,23/12/2016,Washington_Post,https://www.washingtonpost.com/national/for-wo...
195,49292,27/12/2016,Washington_Post,https://www.washingtonpost.com/lifestyle/magaz...
196,49542,29/12/2016,Washington_Post,https://www.washingtonpost.com/news/sports/wp/...


In [74]:
#which to still scrape

#df_empty = df[df['Text']==''].reset_index()
# rescrape the ''1\nComments\n'' text ones
# df_weird = df[df['Text']=='1\nComments\n'].reset_index()
# df_weird
#df_empty

,Title,Text,Date,News Paper,Link
7,,,29/02/2016,Washington_Post,https://www.washingtonpost.com/local/public-sa...
8,By Kristen Hartke,,29/02/2016,Washington_Post,https://www.washingtonpost.com/national/health...
9,,,29/02/2016,Washington_Post,https://www.washingtonpost.com/sports/highscho...
11,This is what it's like to bake brownies when y...,,29/02/2016,Washington_Post,https://www.washingtonpost.com/national/health...
12,"By Gregory Lee Sullivan\nFebruary 29, 2016",,29/02/2016,Washington_Post,https://www.washingtonpost.com/national/health...
...,...,...,...,...,...
49578,,,01/01/2016,Washington_Post,https://www.washingtonpost.com/entertainment/b...
49580,It’s high season for online dating — plot your...,,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/soloish/wp...
49582,Odell Beckham Jr. vows to ‘learn’ from suspens...,,01/01/2016,Washington_Post,https://www.washingtonpost.com/news/early-lead...
49583,Correction: An earlier version of this story h...,,01/01/2016,Washington_Post,https://www.washingtonpost.com/local/dc-marche...


#which to still scrape
year = "2016"
#source = 'Washington_Post_'
source = "Wall_Street_Journal_"
original_articles = pd.read_parquet(source+year+"_Articles")
articles = pd.read_parquet(source+year+"_Articles")
urls= pd.read_parquet(source+year+"_URLS")



len(URLS_WP)
articles

articles

In [14]:
sum(articles.duplicated(keep='last', subset = ['Link']))

51

In [30]:
sum(articles['Title']== "404")
len(articles)

36428

## 3.3. New York Times

## 4. Clean scrapet articles

# 4. Analysing Scraping Results

## 4.1. Load in all Files

### Wall Street Journal

In [28]:
# Set the directory path
directory = "C:/Users/Boedt/OneDrive/Bureaublad/Scraped_Articles/Wall Street Journal"

# Get a list of filenames in the directory
filenames = os.listdir(directory)

# Loop through the filenames and read each Parquet file into a DataFrame
dfs = []
for filename in filenames:
    filepath = os.path.join(directory, filename)
    df = pd.read_parquet(filepath)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
WSJ_df = pd.concat(dfs, ignore_index=True)

In [29]:
WSJ_df.head(5)

,Title,Text,Link,Date,News Paper,index
0,EU Approves Dell’s Takeover of EMC,BRUSSELS—The European Union on Monday approve...,http://www.wsj.com/articles/eu-approves-dells-...,29/02/2016,Wall_Street_Journal,NaN
1,Treasurys Headed for Another Monthly Gain,While Treasurys were boosted overnight by a 2...,http://www.wsj.com/articles/demand-for-austral...,29/02/2016,Wall_Street_Journal,NaN
2,Citi Reaches Deal for American Express’s Costc...,Citigroup Inc. C 1.18% reached a deal to buy ...,http://www.wsj.com/articles/citi-reaches-deal-...,29/02/2016,Wall_Street_Journal,NaN
3,Jeff Immelt: GE ‘Underowned’ by Big Investors,Jeff Immelt says General Electric Co. GE 0.93...,http://www.wsj.com/articles/jeff-immelt-ge-und...,29/02/2016,Wall_Street_Journal,NaN
4,New Eaton Vance Fund Launches but Isn’t Widely...,Eaton Vance Corp. hasn’t signed any major sec...,http://www.wsj.com/articles/new-eaton-vance-fu...,29/02/2016,Wall_Street_Journal,NaN


In [30]:
#set type of the date column to date
WSJ_df["Date"] = pd.to_datetime(WSJ_df['Date'], format='%d/%m/%Y')

In [31]:
#Remove the index column
WSJ_df = WSJ_df.drop("index", axis = 1)

### Washington Post

In [4]:
# Set the directory path
directory = "C:/Users/Boedt/OneDrive/Bureaublad/Scraped_Articles/Washington Post"

# Get a list of filenames in the directory
filenames = os.listdir(directory)

# Loop through the filenames and read each Parquet file into a DataFrame
dfs = []
for filename in filenames:
    filepath = os.path.join(directory, filename)
    df = pd.read_parquet(filepath)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
WP_df = pd.concat(dfs, ignore_index=True)

In [5]:
WP_df.head(5)

,Title,Text,Date,News Paper,Link
0,Daniel Winnik was ‘shocked’ to be traded to Ca...,"Around 10:45 p.m., a Toronto number Daniel Win...",29/02/2016,Washington_Post,https://www.washingtonpost.com/news/capitals-i...
1,JSOC commander tapped to lead Special Operatio...,Lt. Gen. Raymond “Tony” Thomas has been offici...,29/02/2016,Washington_Post,https://www.washingtonpost.com/news/checkpoint...
2,Chris Christie is setting himself up to be Rep...,Chris Christie is getting savaged for becoming...,29/02/2016,Washington_Post,https://www.washingtonpost.com/news/the-fix/wp...
3,Lawsuit takes aim at asset forfeiture in Indiana,"More here. Under the law, law enforcement agen...",29/02/2016,Washington_Post,https://www.washingtonpost.com/news/the-watch/...
4,Holly Holm says she won’t chase money with Ron...,Holly Holm fully understands that the most pro...,29/02/2016,Washington_Post,https://www.washingtonpost.com/news/early-lead...


In [6]:
#set type of the date column to date
WP_df["Date"] = pd.to_datetime(WP_df['Date'], format='%d/%m/%Y')

## 4.2. Clean URLs

### Wall Street Journal

In [32]:
WSJ_duplicate_URLs = WSJ_df['Link'].value_counts()
WSJ_duplicate_URLs = WSJ_duplicate_URLs[WSJ_duplicate_URLs > 1].reset_index()
WSJ_duplicate_URLs.columns = ['Link', 'Count']

In [37]:
WSJ_duplicate_URLs["Link"][5]

'https://www.wsj.com/articles/why-apple-and-the-carriers-want-your-old-iphone-4279cc8d?mod=error_page'

In [38]:
WSJ_df[WSJ_df["Link"] == WSJ_duplicate_URLs["Link"][5]]

,Title,Text,Link,Date,News Paper


In [35]:
#Drop all the duplicate rows
WSJ_df = WSJ_df.drop_duplicates().reset_index(drop = True)

In [36]:
#Some URLs of the Wall Street Journal contain the string "mod=error_page". Once we look into this we see that these URLs lead us
#to articles that where published on a date outside the scope. We assume that these were URLs where something went wrong during
#the scraping process and thus we received an error URL instead. We first remove these URLs from the dataframe
mask = ~ WSJ_df['Link'].str.contains('mod=error_page')
WSJ_df = WSJ_df[mask].reset_index(drop = True)

### Washington Post

In [7]:
#Drop all duplicates
WP_df = WP_df.drop_duplicates().reset_index(drop = True)

In [8]:
WP_duplicate_URLs = WP_df['Text'].value_counts()
WP_duplicate_URLs = WP_duplicate_URLs[WP_duplicate_URLs > 1].reset_index()
WP_duplicate_URLs.columns = ['Link', 'Count']

In [9]:
WP_duplicate_URLs[0:10]

,Link,Count
0,,33779
1,Photo,3714
2,0\nComments\n,1668
3,1\nComments\n,683
4,2\nComments\n,556
5,3\nComments\n,454
6,"Opinions to start the day, in your inbox. Sign...",416
7,2021 Election: Complete coverage and analysis\n,401
8,4\nComments\n,339
9,5\nComments\n,297


In [11]:
output = WP_df[(WP_df["Text"].str.split(" ").str.len() == 1) & (WP_df["Text"].str.contains("Comments"))].reset_index(drop = True)

In [16]:
list(output["Link"])[0]

'https://www.washingtonpost.com/opinions/is-gmo-opposition-immoral/2018/01/01/2c9e6a54-ecc3-11e7-956e-baea358f9725_story.html'

In [27]:
WP_df[WP_df["Link"] == list(output["Link"])[12903]]

,Title,Text,Date,News Paper,Link
150038,‘Negotiating Public Services in the Congo’ is ...,0\nComments\n,2020-06-26,Washington_Post,https://www.washingtonpost.com/politics/2020/0...


## 4.3. Unique URLs

### Wall Street Journal

In [39]:
#First we create a two new columns with the year, month of the articles -> this way we can look at the amount of URLs
#per year and per month

WSJ_df["Year"] = WSJ_df['Date'].dt.year
WSJ_df["Month"] = WSJ_df["Date"].dt.month

In [40]:
WSJ_df.head(3)

,Title,Text,Link,Date,News Paper,Year,Month
0,EU Approves Dell’s Takeover of EMC,BRUSSELS—The European Union on Monday approve...,http://www.wsj.com/articles/eu-approves-dells-...,2016-02-29,Wall_Street_Journal,2016,2
1,Treasurys Headed for Another Monthly Gain,While Treasurys were boosted overnight by a 2...,http://www.wsj.com/articles/demand-for-austral...,2016-02-29,Wall_Street_Journal,2016,2
2,Citi Reaches Deal for American Express’s Costc...,Citigroup Inc. C 1.18% reached a deal to buy ...,http://www.wsj.com/articles/citi-reaches-deal-...,2016-02-29,Wall_Street_Journal,2016,2


In [41]:
#Unique URLs per year -> same article on different date is unique URL
WSJ_URLs_Year = WSJ_df.groupby(["News Paper","Year", "Date"])["Link"].nunique().reset_index(name='Unique_URLs_Count')
WSJ_URLs_Year.groupby("Year").sum("Unique_URLs_Count")

,Unique_URLs_Count
Year,
2016,58736
2017,49757
2018,28891
2019,36185
2020,32595


In [42]:
#Unique URLs per year -> same article on different date is no unique URL
WSJ_URLs_Year = WSJ_df.groupby(["News Paper","Year"])["Link"].nunique().reset_index(name='Unique_URLs_Count')
WSJ_URLs_Year

,News Paper,Year,Unique_URLs_Count
0,Wall_Street_Journal,2016,58736
1,Wall_Street_Journal,2017,49754
2,Wall_Street_Journal,2018,28891
3,Wall_Street_Journal,2019,36178
4,Wall_Street_Journal,2020,32594


In [43]:
#Unique URLs per year
WSJ_URLs_Month = WSJ_df.groupby(["News Paper","Year", "Month"])["Link"].nunique().reset_index(name='Unique_URLs_Count')
WSJ_URLs_Month

,News Paper,Year,Month,Unique_URLs_Count
0,Wall_Street_Journal,2016,2,54
1,Wall_Street_Journal,2016,3,5203
2,Wall_Street_Journal,2016,4,5699
3,Wall_Street_Journal,2016,5,6372
4,Wall_Street_Journal,2016,6,6877
5,Wall_Street_Journal,2016,7,6438
6,Wall_Street_Journal,2016,8,6350
7,Wall_Street_Journal,2016,9,6297
8,Wall_Street_Journal,2016,10,6389
9,Wall_Street_Journal,2016,11,5960


### Washington Post

In [118]:
#First we create a two new columns with the year, month of the articles -> this way we can look at the amount of URLs
#per year and per month

WP_df["Year"] = WP_df['Date'].dt.year
WP_df["Month"] = WP_df["Date"].dt.month

In [119]:
WP_df.head(3)

,Title,Text,Date,News Paper,Link,Year,Month
0,Daniel Winnik was ‘shocked’ to be traded to Ca...,"Around 10:45 p.m., a Toronto number Daniel Win...",2016-02-29,Washington_Post,https://www.washingtonpost.com/news/capitals-i...,2016,2
1,JSOC commander tapped to lead Special Operatio...,Lt. Gen. Raymond “Tony” Thomas has been offici...,2016-02-29,Washington_Post,https://www.washingtonpost.com/news/checkpoint...,2016,2
2,Chris Christie is setting himself up to be Rep...,Chris Christie is getting savaged for becoming...,2016-02-29,Washington_Post,https://www.washingtonpost.com/news/the-fix/wp...,2016,2


In [120]:
#Unique URLs per year
WP_URLs_Year = WP_df.groupby(["News Paper","Year"])["Link"].nunique().reset_index(name='Unique_URLs_Count')
WP_URLs_Year

,News Paper,Year,Unique_URLs_Count
0,Washington_Post,2016,49664
1,Washington_Post,2017,49800
2,Washington_Post,2018,27888
3,Washington_Post,2019,10233
4,Washington_Post,2020,13697


In [ ]:
#Unique URLs per year
WP_URLs_Month = WP_df.groupby(["News Paper","Year", "Month"])["Link"].nunique().reset_index(name='Unique_URLs_Count')
#WP_URLs_Month

## 4.4. Clean Articles

### Wall Street Journal

#### Remove all articles which we don't have access to

In [ ]:
WSJ_df = pd.read_parquet("WSJ_missed_articles")

In [16]:
#Remove all articles that have the text "NO ACCESS" -> we didn't have access to these
remove_NO_ACCESS = ~ WSJ_df["Text"].str.contains("NO ACCESS")
WSJ_df = WSJ_df[remove_NO_ACCESS].reset_index(drop = True)

In [17]:
WSJ_df.drop_duplicates().shape[0]

35614

In [18]:
WSJ_df.nunique()

Title         18844
Text          18720
Link          31652
Date            661
News Paper        1
Year              5
dtype: int64

#### Empty text

Articles where we didn't collect text -> was either rescraped, or there was no text to begin with

In [19]:
#Remove all blank space in the text
remove_blank_text_df = WSJ_df
remove_blank_text_df["Text"] = remove_blank_text_df['Text'].str.strip()

In [20]:
WSJ_df = WSJ_df.drop(remove_blank_text_df[remove_blank_text_df["Text"] == ""].index)

In [21]:
WSJ_df.drop_duplicates().shape[0]

19292

In [22]:
WSJ_df.nunique()

Title         18483
Text          18715
Link          19129
Date            560
News Paper        1
Year              5
dtype: int64

#### Other text that failed to be collected or is not usefull

In [ ]:
#These articles text only contains the end of the article, but we didn't scrape the article itself
WSJ_df = WSJ_df[~WSJ_df["Text"].str.startswith('Write to')].reset_index(drop = True)

In [ ]:
WSJ_df.drop_duplicates().shape[0]

In [ ]:
WSJ_df.nunique()

In [ ]:
#Some articles had a PDF you had to download
WSJ_df = WSJ_df[WSJ_df["Text"] != "Download PDF"].reset_index(drop = True)
WSJ_df = WSJ_df[WSJ_df["Text"] != "See Solution Download PDF"].reset_index(drop = True)
WSJ_df = WSJ_df[WSJ_df["Text"] != "Download PDF See Solution"].reset_index(drop = True)

### Washington Post

In [ ]:
WSJ_df.groupby(["News Paper","Year", "Date"])["Link"].nunique().reset_index(name='Unique_URLs_Count').groupby("Year").sum("Unique_URLs_Count")

## 4.5. Articles

### Wall Street Journal

In [27]:
#cleaning first scrape

WSJ_df = pd.read_parquet("WSJ_missed_articles")
#Remove all articles that have the text "NO ACCESS" -> we didn't have access to these
remove_NO_ACCESS = ~ WSJ_df["Text"].str.contains("NO ACCESS")
WSJ_df = WSJ_df[remove_NO_ACCESS].reset_index(drop = True)
WSJ_df.drop_duplicates().shape[0]
WSJ_df.nunique()

Title         18844
Text          18720
Link          31652
Date            661
News Paper        1
Year              5
dtype: int64

In [30]:
#### Empty text
#Articles where we didn't collect text -> was either rescraped, or there was no text to begin with
#Remove all blank space in the text
remove_blank_text_df = WSJ_df
remove_blank_text_df["Text"] = remove_blank_text_df['Text'].str.strip()
WSJ_df = WSJ_df.drop(remove_blank_text_df[remove_blank_text_df["Text"] == ""].index).reset_index(drop=True)
WSJ_df.drop_duplicates().shape[0]
WSJ_df.nunique()

Title         18483
Text          18715
Link          19129
Date            560
News Paper        1
Year              5
dtype: int64

In [32]:
WSJ_df
URL = pd.read_parquet("WSJ_missed_URLs")
URL

,Date,News Paper,Link,Year
0,2016-06-16,Wall_Street_Journal,http://www.wsj.com/articles/phil-mickelson-and...,2016
1879,2016-12-29,Wall_Street_Journal,http://www.wsj.com/articles/what-1917-set-in-m...,2016
1880,2016-12-29,Wall_Street_Journal,http://www.wsj.com/articles/ukraine-must-make-...,2016
1882,2016-12-29,Wall_Street_Journal,http://www.wsj.com/articles/faith-that-upholds...,2016
1883,2016-12-29,Wall_Street_Journal,http://www.wsj.com/articles/free-speech-and-an...,2016
...,...,...,...,...
35328,2020-05-15,Wall_Street_Journal,https://www.wsj.com/articles/coronavirus-pande...,2020
35329,2020-05-29,Wall_Street_Journal,https://www.wsj.com/articles/weve-hardly-gotte...,2020
35330,2020-06-01,Wall_Street_Journal,https://www.wsj.com/articles/pep-talk-for-poli...,2020
35331,2020-06-03,Wall_Street_Journal,https://www.wsj.com/articles/he-knew-george-fl...,2020


In [39]:
WSJ_df.to_parquet('WSJ_missed_articlesV2')

In [37]:
df = pd.merge(URL, WSJ_df, on=['Link', 'Date', 'News Paper', 'Year'], how="outer", indicator=True).query('_merge=="left_only"').reset_index(drop= True).drop(['Title', 'Text', '_merge'], axis=1)
df
df.to_parquet('WSJ_missed_URLsV2')